In [ ]:
!pip install install-jdk
!pip install pyspark --user
!pip install findspark --user

In [ ]:
import jdk
jdk.install('11')

In [ ]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os
os.environ["JAVA_HOME"] = "/home/jovyan/.jdk/jdk-11.0.18+10"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.8"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3.8"
findspark.init()
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('sparktest').setMaster('k8s://https://kubernetes.default.svc:443')
conf.set("spark.kubernetes.namespace", "kubeflow-user-example-com")
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "default-editor")

conf.set("spark.kubernetes.container.image", "lexcc/spark-py")
conf.set("spark.kubernetes.executor.container.image", "lexcc/spark-py")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")

conf.set("spark.kubernetes.allocation.batch.size", "5")
conf.set("spark.kubernetes.executor.instances", "1")
conf.set("spark.driver.bindAddress", "0.0.0.0")
conf.set("spark.driver.host", "jupyter")
conf.set("spark.driver.port", "37371")
conf.set("spark.blockManager.port", "6060")

conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.2')

In [ ]:
spark = SparkSession.builder.config(conf=conf).appName("Word Count").getOrCreate()
sc = spark.sparkContext

In [ ]:
sc.parallelize([1,2,3,4,5]).count()

In [ ]:
import random
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = sc.parallelize(range(0, 100)) \
             .filter(inside).count()
print("Pi is roughly %f" % (4.0 * count / 100))

In [ ]:
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minio")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minio123")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "minio-service.kubeflow:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

In [ ]:
df = sc.textFile("s3a://mlpipeline/account-containers-list-response.json")

In [ ]:
df.collect()
#df.write.format("csv").option("header", "true").save("s3a://datasets/result")